In [169]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

import warnings as w
w.filterwarnings('ignore')

In [170]:
# read the training document
df = pd.read_csv("train.csv", error_bad_lines=False, encoding='utf-8')

In [171]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [172]:
df.shape

(20800, 5)

In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [174]:
## Drop Null values
df.dropna(inplace=True)

In [175]:
df.shape

(18285, 5)

In [176]:
df[df['label']==1]

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
12,12,Russian Researchers Discover Secret Nazi Milit...,Amando Flavio,The mystery surrounding The Third Reich and Na...,1
...,...,...,...,...,...
20788,20788,Maine’s Gov. LePage Threatens To ‘Investigate’...,Joe Clark,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
20791,20791,Lawyer Who Kept Hillary Campaign Chief Out of ...,Daniel Greenfield,Lawyer Who Kept Hillary Campaign Chief Out of ...,1
20793,20793,Idiot Who Destroyed Trump Hollywood Star Gets ...,Robert Rich,Share This \nAlthough the vandal who thought i...,1
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [177]:
df.label.value_counts(normalize=True)*100

0    56.663932
1    43.336068
Name: label, dtype: float64

It is a Balanced dataset

In [178]:
## Fetch Independent feature

X = df.drop('label',axis=1)

In [179]:
## Fetch dependent variable
y = df['label']

In [180]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [181]:
tf.__version__

'2.12.0'

In [182]:
## Setting Vocabulary Size
voc_size = 5000

## One Hot Representation

In [183]:
news = X.copy()

In [184]:
news['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [185]:
news.reset_index(inplace = True)

In [186]:
news.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [187]:
import nltk
import re
from nltk.corpus import stopwords

In [188]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Data Preprocessing

In [127]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(news)):
  review = re.sub('[^a-zA-Z]', ' ', news['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [128]:
corpus[5]

'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart'

## One Hot Representation

In [129]:
## Assign index to words in corpus with vocab size not greater than 5000
onehot = [one_hot(words, voc_size) for words in corpus]
onehot[1]

[1305, 2540, 1531, 2336, 4345, 3381, 3977]

## Embedding Representation

In [168]:
## make all sentences of length of 20
senten_len = 20
embedded_docs = pad_sequences(onehot, padding = 'post',maxlen = senten_len)
embedded_docs[11]

array([ 173, 2011, 4470, 2979, 2707, 1459,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

## Creating Model

In [131]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = senten_len))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='cross_binaryentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_6 (LSTM)               (None, 100)               56400     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [132]:
embedding_vector_features = 40
model1 = Sequential()
model1.add(Embedding(voc_size, embedding_vector_features, input_length = senten_len))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1, activation = 'sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_5 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [138]:
len(embedded_docs),y.shape

(18285, (18285,))

In [139]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [140]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [141]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [142]:
y_train[1]

1

## Model Training

In [143]:
model1.fit(X_train,y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 23s 91ms/step - loss: 0.2970 - accuracy: 0.8622 - val_loss: 0.2020 - val_accuracy: 0.9147
Epoch 2/10
192/192 [==============================] - 17s 87ms/step - loss: 0.1373 - accuracy: 0.9480 - val_loss: 0.2368 - val_accuracy: 0.9123
Epoch 3/10
192/192 [==============================] - 19s 99ms/step - loss: 0.0935 - accuracy: 0.9664 - val_loss: 0.2625 - val_accuracy: 0.9087
Epoch 4/10
192/192 [==============================] - 16s 84ms/step - loss: 0.0684 - accuracy: 0.9764 - val_loss: 0.3000 - val_accuracy: 0.9042
Epoch 5/10
192/192 [==============================] - 18s 92ms/step - loss: 0.0519 - accuracy: 0.9824 - val_loss: 0.3780 - val_accuracy: 0.9007
Epoch 6/10
192/192 [==============================] - 18s 92ms/step - loss: 0.0363 - accuracy: 0.9884 - val_loss: 0.3553 - val_accuracy: 0.9016
Epoch 7/10
192/192 [==============================] - 19s 99ms/step - loss: 0.0265 - accuracy: 0.9921 - val_loss: 0.4441 - val_accuracy:

## Performance Metrics

In [146]:
y_pred = model1.predict(X_test)

189/189 [==============================] - 4s 15ms/step


In [148]:
y_pred

array([[9.9988467e-01],
       [2.3640369e-03],
       [4.6019970e-05],
       ...,
       [9.1017073e-06],
       [9.9851811e-01],
       [9.9998641e-01]], dtype=float32)

In [158]:
y_test

array([1, 0, 0, ..., 0, 1, 1])

In [159]:
threshold = 0.5

binary_predictions = (y_pred >= threshold).astype(int)

In [150]:
from sklearn.metrics import confusion_matrix

In [160]:
confusion_matrix(y_test, binary_predictions)

array([[3108,  311],
       [ 273, 2343]])

In [164]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_test,binary_predictions)
precision = precision_score(y_test, binary_predictions)
recall = recall_score(y_test, binary_predictions)
f1 = f1_score(y_test, binary_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.9032311516155758
Precision: 0.8828183873398644
Recall: 0.8956422018348624
F1-score: 0.8891840607210627
